In [3]:
from matplotlib import pyplot as plt
import numpy as np

test_point_1 = (0,0)
test_point_2 = (1,1)

# General Tools

This block of code has functions for distance, magnitude or length, midpoint calculation, and the warping functions.

In [51]:
def Distance(p_1, p_2):
    p_1x, p_1y = p_1
    p_2x, p_2y = p_2

    return np.sqrt((p_2x-p_1x)**2 + (p_2y-p_1y)**2)

def Length(point):
    return Distance(point, (0,0))

def Midpoint(p_1, p_2):
    x_1, y_1 = p_1
    x_2, y_2 = p_2

    midpoint_x = 0.5*(x_1+x_2)
    midpoint_y = 0.5*(y_1+y_2)

    midpoint = (midpoint_x, midpoint_y)

    return midpoint


def Complex_Function(point):
    p_x, p_y = point
    output_x = p_x/(Length(point)**2)
    output_y = -p_y/(Length(point)**2)

    return (output_x, output_y)

def Stretch_Swirl_Function(center_point, input_point,coeff_stretch, coeff_swirl):
    in_x, in_y = input_point
    points_distance = Distance(center_point, input_point)

    modifier_stretch = (points_distance**2 + 1)/coeff_stretch
    modifier_swirl = coeff_swirl/(points_distance**2 + 1)

    out_x = modifier_stretch*(np.cos(modifier_swirl)*in_x + np.sin(modifier_swirl)*in_y)
    out_y = modifier_stretch*(-np.sin(modifier_swirl)*in_x + np.cos(modifier_swirl)*in_y)

    return (out_x, out_y)             

# Grid and Edges

Here the grid of points is generated along with the class "Puzzle_Edge" is defined. Puzzle_Edge is a class to defined to have the end points of the puzzle edge, find the midpoint, and then set two spots for the "anchor points". The anchor points will be calculated later on to create the jigsaw shape with the NURB spline.

In [65]:
def Grid_Points(number_points):
    input_x_value = np.arange(-number_points, number_points)
    input_y_value = np.arange(-number_points, number_points)

    input_points = [(i,j) for i in input_x_value for j in input_y_value]
    return input_points

class Puzzle_Edge:
    def __init__(self, lesser_endpoint=None, greater_endpoint=None):
        self.lesser_endpoint = lesser_endpoint
        self.lesser_anchor = None
        self.midpoint = Midpoint(lesser_endpoint, greater_endpoint)
        self.lesser_anchor = None
        self.greater_endpoint = greater_endpoint

def Horizontal_Edges(points):
    horizontal_edges = []
    for point in points:
        h_edge = Puzzle_Edge(point,(point[0]+1,point[1]))
        horizontal_edges += [h_edge]
    return horizontal_edges

def Vertical_Edges(points):
    vertical_edges = []
    for point in points:
        v_edge = Puzzle_Edge(point,(point[0],point[1]+1))
        vertical_edges += [v_edge]
    return vertical_edges
    

In [64]:
test_grid = Grid_Points(2)
test_horizontal_edges = Horizontal_Edges(test_grid)
test_vertical_edges = Vertical_Edges(test_grid)

In [96]:
def Uniform_Knot_Vector(number_points, order):
    return np.arange(number_points + order + 1)

def Rising_Basis(knot_vector, index, order, time):
    if (knot_vector[index] == knot_vector[index + order]):
        return 0
    return (time - knot_vector[index])/(knot_vector[index+order] - knot_vector[index])


def Falling_Basis(knot_vector, index, order, time):
    return 1-Rising_Basis(knot_vector, index, order, time)

def N_Basis(knot_vector, index, order, time):
    if (order == 0):
        if (knot_vector[index] <= time <= knot_vector[index+1]):
            return 1
        else:
            return 0
    else:
        rising_output = Rising_Basis(knot_vector, index, order, time)*N_Basis(knot_vector, index, order-1, time)
        falling_output = Falling_Basis(knot_vector, index+1, order, time)*N_Basis(knot_vector, index+1, order-1, time)
        return rising_output + falling_output

def NURBS_Coefficients(knot_vector, weights, index, order, time):
    length = len(weights)
    normalize_constant = sum([N_Basis(knot_vector, j, order, time)*weight for j, weight in zip(range(length),weights)])

    return N_Basis(knot_vector, index,order,time)*weights[index]/normalize_constant

def NURBS(points,order,weights):
    knot_vector = np.arange(len(points) + order + 1)
    